In [322]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Загружаем данные по датасету

In [323]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train1 = pd.read_csv(DATA_DIR +'/train.csv')
df_test1 = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

Для корректной обработки признаков объединяем трейн и тест в один датасет

In [324]:
df_train1['sample'] = 1 # помечаем где у нас трейн
df_test1['sample'] = 0  # помечаем где у нас тест
df_test1['default'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data1 = df_test1.append(df_train1, sort=False).reset_index(drop=True) # объединяем

Посмотрим на датасет, проверим пропуски

In [325]:
data1.info()

Пропуски в столбце Образование. Для корректной работы модели и последующей валидации не удаляем их, заменим пропуски на No_data

In [326]:
data1['education'] = data1['education'].fillna(value = 'No_data')

Проверяем

In [327]:
data1['education'].value_counts(dropna = False)

Пропусков больше нет. Проверим наш таргет:

In [328]:
data1['default'].value_counts()

Количество "недефолтов" в 10 раз больше дефолтов, если просто будем использовать объединенный датасет модель будет ставить отсутствие дефолта в большинстве случаев. Поэтому балансируем выборку: берем все дефолты, первые 9372 записей с "недефолтом" и все тестовые данные, которые удалим при обучении (они должны быть обработаны в соответствии с обучающей выборкой).

In [329]:
vis_data_1 = data1[data1['default'] == 1].copy()
vis_data_1.shape

In [330]:
vis_data_3 = data1[data1['sample'] == 0].copy()
vis_data_3.shape

In [331]:
temp_data = data1[data1['default'] == 0].copy()
temp_data1 = temp_data[temp_data['sample'] == 1].copy()

In [332]:
vis_data_2 = temp_data1.iloc[:9372,:].copy()
vis_data_2.shape

In [333]:
data_temp = pd.concat([vis_data_2,vis_data_1], axis = 0, ignore_index = True)
data_sampled = pd.concat([vis_data_3,data_temp], axis = 0, ignore_index = True)
data_sampled.shape

Подготовили датасет для последующей обработки, посмотрим на него:

In [334]:
data_sampled

Видим 3 типа данных: категориальные, числовые, бинарные. Промаркируем их для удобства при последующей обработке

In [335]:
num_cols1 = ['age', 'score_bki', 'income', 'decline_app_cnt', 'bki_request_cnt']
cat_cols1 = ['education', 'first_time', 'sna', 'home_address','work_address', 'region_rating']
bin_cols1 = ['sex' ,'car', 'car_type', 'good_work', 'foreign_passport']

Удалим столбцы с ID и датой

In [336]:
data_sampled.drop(['client_id','app_date',], axis = 1, inplace=True)

Построим матрицу корреляций

In [337]:
corr = data_sampled.corr()
#
# Set up the matplotlib plot configuration
#
f, ax = plt.subplots(figsize=(20, 10))
#
# Generate a mask for upper traingle
#
mask = np.triu(np.ones_like(corr, dtype=bool))
#
# Configure a custom diverging colormap
#
cmap = sns.diverging_palette(230, 30, as_cmap=True)
#
# Draw the heatmap
#
sns.heatmap(corr, annot=True, mask = mask, cmap=cmap)

Видим, что самое значительное влияние на таргет оказывает score_bki (sample - искуственный признак, который потом удалим). Видим значительную корреляцию между машиной и типом машины, между домашним и рабочим адресом.

In [338]:
data_sampled['score_bki'].describe()

Можно было бы удалить car и work_address из-за высокой корреляции с типом машины и рабочим адресом, но это негативно повлияло на результаты логрегресии, поэтому не применяем строчку с кодом ниже, идем дальше

In [339]:
#data_sampled.drop(['car','work_address',], axis = 1, inplace=True)

Также была попытка улучшить распределение score_bki, убрав выбросы по межквартильному размаху, но это опять ухудшило результаты логрегресии, не трогаем этот признак, идем дальше

In [340]:
d = data_sampled['score_bki'].median()

In [341]:

#определим интерквартильное расстояние
IQR = data_sampled['score_bki'].quantile(0.75) - data_sampled['score_bki'].quantile(0.25)
perc25 = data_sampled['score_bki'].quantile(0.25)
perc75 = data_sampled['score_bki'].quantile(0.75)

print(
    '25-й перцентиль: {},'.format(perc25),
    '75-й перцентиль: {},'.format(perc75),
    "IQR: {}, ".format(IQR),
    "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

data_sampled['score_bki'].loc[data_sampled['score_bki'].between(
    perc25 - 0.5*IQR,
    perc75 + 0.5*IQR)].hist(bins=10, range=(-5, 1), label='IQR')

data_sampled['score_bki'].loc[data_sampled['score_bki'] <= 1].hist(
    alpha=0.5, bins=10, range=(-5, 1), label='Выбросы')

plt.legend()

In [342]:
#data_sampled['score_bki'] = data_sampled['score_bki'].apply(lambda x: d if x < (perc25 - 1*IQR) or x > (perc75 + 1*IQR) else x)

Попытка добавить новый признак на основе score_bki также не увенчалась успехом, пропускаем

In [343]:
#data_sampled['feature'] = data_sampled['score_bki']*data_sampled['age']

Поменяем знак для score_bki для того, чтобы потом его прологарифмировать

In [344]:
data_sampled['score_bki'] = data_sampled['score_bki'].apply(lambda x: x*(-1) if x <0 else x)

Посмотрим на распределение числовых признаков

In [345]:
for i in num_cols1:
    plt.figure()
    sns.distplot(data_sampled[i].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

Все признаки, кроме score_bki распределены не нормально. Попытки улучшить распределение bki_request_cnt и decline_app_cnt ухудшили результаты модели, т.к. большое количество значений с нулями, изменение которых влияет потом на всю модель, поэтому эти два признака не трогаем, возраст и доход логарифмируем.

In [346]:
#data_sampled['bki_request_cnt'] = data_sampled['bki_request_cnt'].apply(lambda x: 5 if x > 5 else x)
#data_sampled['decline_app_cnt'] = data_sampled['decline_app_cnt'].apply(lambda x: 1 if x > 1 else x)

In [347]:
#k = data_sampled['feature'].median()
#IQR = data_sampled['feature'].quantile(0.75) - data_sampled['feature'].quantile(0.25)
#perc25 = data_sampled['feature'].quantile(0.25)
#perc75 = data_sampled['feature'].quantile(0.75)

#print(
   # '25-й перцентиль: {},'.format(perc25),
    #'75-й перцентиль: {},'.format(perc75),
   # "IQR: {}, ".format(IQR),
   # "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

#data_sampled['feature'].loc[data_sampled['feature'].between(
    #perc25 - 0.5*IQR,
    #perc75 + 0.5*IQR)].hist(bins=30, range=(-250, 1), label='IQR')

#data_sampled['feature'].loc[data_sampled['feature'] <= 1].hist(
   #alpha=0.5, bins=30, range=(-250, 1), label='Выбросы')

#plt.legend()

In [348]:
#data_sampled['feature'] = data_sampled['feature'].apply(lambda x: k if x < (perc25 - 1*IQR) or x > (perc75 + 1*IQR) else x)

In [349]:
#data_sampled['feature'].hist()

In [350]:
#data_sampled['decline_app_cnt'].value_counts()

In [351]:
from scipy.stats import lognorm

Логарифмируем данные, включая score_bki

In [352]:
data_sampled['age'] = np.log2(data_sampled['age'])
data_sampled['income'] = np.log(data_sampled['income'])
data_sampled['score_bki'] = np.log(data_sampled['score_bki'])

Посмотрим теперь на их распределение

In [353]:
for i in num_cols1:
    plt.figure()
    sns.distplot(data_sampled[i].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

С доходом все прекрасно, у возраста улучшилось распределение (можно улучшить еще, но после многочасовых попыток это ни к чему не привело)

In [354]:
from sklearn.feature_selection import f_classif

Посмотрим еще раз какой признак сильнее всего влияет на целевую переменную

In [355]:
imp_num = pd.Series(f_classif(data_sampled[num_cols1], data_sampled['default'])[0], index = num_cols1)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [356]:
data_sampled.boxplot('score_bki','default')

У людей с дефолтом в среднем рейтинг ниже

In [357]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

Вместо get_dummies применим label encoder для бинарно-категориальных столбцов

In [358]:
for column in bin_cols1:
    data_sampled[column] = label_encoder.fit_transform(data_sampled[column])
    
# убедимся в преобразовании    
data_sampled.head()

In [359]:
data_sampled.info()

Все данные в цифровом формате, кроме education, исправим это:

In [360]:
mapped_education = pd.Series(label_encoder.fit_transform(data_sampled['education']))
print(dict(enumerate(label_encoder.classes_)))

In [361]:
data_sampled['education'] = label_encoder.fit_transform(data_sampled['education'])

In [362]:
data_sampled

Все ок, можно строить модель

In [363]:
# Теперь выделим тестовую часть
train_data1 = data_sampled.query('sample == 1').drop(['sample'], axis=1)
test_data1 = data_sampled.query('sample == 0').drop(['sample', 'default'], axis=1)

y1 = train_data1['default'].values  # наш таргет
X1 = train_data1.drop(['default'], axis=1)

In [364]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
from sklearn.model_selection import train_test_split

# выделим 20% данных на валидацию (параметр test_size)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [365]:
from sklearn.linear_model import LogisticRegression

In [366]:
logreg1 = LogisticRegression(solver='liblinear', max_iter=5000)
logreg1.fit(X_train1, y_train1)
y_pred1 = logreg1.predict(X_test1)

Здесь сразу вторая логрегрессия с немного другими гиперпараметрами (они выявляются позже по коду)

In [367]:
logreg2 = LogisticRegression(penalty = 'none',solver='lbfgs', max_iter=1000)
logreg2.fit(X_train1, y_train1)
y_pred2 = logreg2.predict(X_test1)

In [368]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test1, y_pred1,digits=5)
print(classification_report)

Средний f1_score лучше бейзлайна (в нем 0,47)

In [369]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test1, y_pred2,digits=5)
print(classification_report)

Вторая модель показывает чуть лучшие результаты, возьмем ее

In [382]:
logreg_final1 = LogisticRegression( penalty = 'none',solver='lbfgs', max_iter=1000)
logreg_final1.fit(X1, y1)

In [383]:
predict_submission = logreg_final1.predict(test_data1)

In [384]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [385]:
sample_submission['default'].value_counts()

Посмотрим на количество предсказанных дефолтов в сабмишн

Подбор гиперпараметров

In [374]:
from sklearn.model_selection import GridSearchCV

In [375]:
model = LogisticRegression()

iter_ = 600
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

## model ваша модель логистической регрессии
gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train1, y_train1)
model = gridsearch.best_estimator_

##печатаем параметры
best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

In [376]:
!kaggle competitions submit -c sf-scoring -f submission.csv -m "Message"